In [1]:
import pandas as pd
import scipy.stats as stats
import os

Concatenate our files. Reminder: we have a file per year, we all of 'Informatique' in it.

In [2]:
# requires os
def concatFiles(direc, fileType):
    files = os.listdir(direc)
    if '.DS_Store' in files:
        files.remove('.DS_Store')
    print(files)
    if fileType == 'csv':
        r = pd.read_csv
    for idx, file in enumerate(files):
        file = direc+file
        if idx == 0:
            df = r(file, header=0)
            print(file)
        else:
            new = r(file, header=0)
            df = pd.concat([df, new], axis=0)
            print(file)
    return df

In [3]:
info = concatFiles('data/', 'csv')

['informatique_2011_2012.csv', 'informatique_2014_2015.csv', 'informatique_2012_2013.csv', 'informatique_2008_2009.csv', 'informatique_2007_2008.csv', 'informatique_2016_2017.csv', 'informatique_2015_2016.csv', 'informatique_2010_2011.csv', 'informatique_2013_2014.csv', 'informatique_2009_2010.csv']
data/informatique_2011_2012.csv
data/informatique_2014_2015.csv
data/informatique_2012_2013.csv
data/informatique_2008_2009.csv
data/informatique_2007_2008.csv
data/informatique_2016_2017.csv
data/informatique_2015_2016.csv
data/informatique_2010_2011.csv
data/informatique_2013_2014.csv
data/informatique_2009_2010.csv


We want to take only the data for the Master students


In [4]:
def locator(s):
    return s.find('Master') != -1

In [6]:
ma = info.loc[info['Semester'].apply(locator)]
ma.head()


,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Subject,Period,Semester
574,Monsieur,Arnfred Jonas,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,184772,Informatique,2011-2012,Master semestre 1
575,Monsieur,Asgari Ehsaneddin,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,211754,Informatique,2011-2012,Master semestre 1
576,Monsieur,Baeriswyl Jonathan,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,179406,Informatique,2011-2012,Master semestre 1
577,Madame,Bai Yi,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,209850,Informatique,2011-2012,Master semestre 1
578,Monsieur,Barroco Michael,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,179428,Informatique,2011-2012,Master semestre 1


We define a function to locate a specific sciper #. They are unique.

In [7]:
def locSciper(df, sciper):
    return df.loc[df['No Sciper'] == sciper]

In [10]:
def getGender(df):
    """ Take a DataFrame and checks the gender
    
    Args:
        df (DataFrame): a DataFrame, typically a .loc on a specific student
        
    Returns:
        bool: True if student is a woman, False otherwise
    """
    try:
        if df.isin(['Madame'])['Civilité'].value_counts()[True] > 0:
            return 1
    except KeyError:
        return 0

In [11]:
def getMaster3(df):
    """ Take a DataFrame and checks if the student had a Master Project
    
    Args:
        df (DataFrame): a DataFrame, typically a .loc on a specific student
        
    Returns:
        bool: True if student had a Master 3, False otherwise
    """
    try:
        if df.isin(['Master semestre 3'])['Semester'].value_counts()[True] > 0:
            return 1
    except KeyError:
        return 0    

In [32]:
def getMasterProject(df):
    """ Take a DataFrame and checks if the student had a Master Project
    
    Args:
        df (DataFrame): a DataFrame, typically a .loc on a specific student
        
    Returns:
        bool: True if student had a Master 3, False otherwise
    """
    try:
        if df.isin(['Projet Master automne'])['Semester'].value_counts()[True] > 0:
            return 1
        elif df.isin(['Projet Master printemps'])['Semester'].value_counts()[True] > 0:
            return 1        
    except KeyError:
        return 0  

In [33]:
def isOneTwo(df):
    """ Take a DataFrame and check that there is an entry for both BA1 and BA6
    
    Args:
        df (DataFrame): a DataFrame, typically a .loc on a specific student
        
    Returns:
        bool: True if it finds both BA1 and BA6, False otherwise
    """
    try:
        one = df.isin(['Master semestre 1'])['Semester'].value_counts()[True] > 0
    except KeyError:
        one = 0
    try:
        two = df.isin(['Master semestre 2'])['Semester'].value_counts()[True] > 0
    except KeyError:
        two = 0
    return (one and two)

Loop through all the students


In [34]:
# We need to remove the students who started too late. 
# Let's say 2 sem without spec. 3 sem with spec

dico = {'sciper': [], 'gender': [], 'length': [], 
        'Master3': [], 'MasterProject': [], 'Specialization':[]}
# Do some tests
for row in ma.iterrows():
    o = row[1]
    sciper = o['No Sciper']  
    if not sciper in dico['sciper']:
        df = locSciper(ma, sciper)
        if isOneTwo(df):
            dico['sciper'].append(sciper)
            dico['gender'].append(getGender(df))
            # Calculating length of stay by nbr of rows
            dico['length'].append(len(df))
            dico['MasterProject'].append(getMasterProject(df))
            dico['Master3'].append(getMaster3(df))
            dico['Specialization'].append(o['Spécialisation'])

In [35]:
data = pd.DataFrame(dico)
data.head()

,Master3,MasterProject,Specialization,gender,length,sciper
0,1,0,NaN,0,3,184772
1,1,0,NaN,0,4,211754
2,1,0,NaN,0,4,179406
3,1,0,NaN,1,3,209850
4,0,0,NaN,0,2,179428


In [36]:
data_spec = data[~data["Specialization"].isnull()]
data_spec.head()

,Master3,MasterProject,Specialization,gender,length,sciper
11,1,0,"Signals, Images and Interfaces",0,4,184991
21,1,0,Internet computing,0,4,214805
24,1,0,Software Systems,0,3,185853
28,1,0,"Signals, Images and Interfaces",0,4,147008
48,1,0,Foundations of Software,0,5,186250


In [38]:
data_spec[data_spec["MasterProject"]==0].head()

,Master3,MasterProject,Specialization,gender,length,sciper
11,1,0,"Signals, Images and Interfaces",0,4,184991
21,1,0,Internet computing,0,4,214805
24,1,0,Software Systems,0,3,185853
28,1,0,"Signals, Images and Interfaces",0,4,147008
48,1,0,Foundations of Software,0,5,186250
